# Juega con los datos en parquet de https://www.kaggle.com/datasets/mkechinov/ecommerce-events-history-in-cosmetics-shop

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

conf = (SparkConf().set("spark.executor.memory", "6g")   #<--- Por executor. Como hay dos instancias, la mitad por executor.
                    .set("spark.executor.cores", "3")    #<--- Por cada executor. Como hay 4, 4*3=12 cores se usan.
                    .set("spark.executor.instances", "2")
                    .set("spark.eventLog.enabled", "true"))
spark = SparkSession.builder.master("spark://spark-master:7077").appName("Ecommerce").config(conf=conf).getOrCreate()
spark

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/11/26 12:32:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
%%time
from pyspark.sql.types import StructType
from pyspark.sql.types import StringType,BooleanType,DateType,IntegerType
from pyspark.sql.functions import substring
import pyspark.sql.functions as F

proc='*'

df = spark.read.option("mergeSchema", True).option("basePath", "file:///opt/workspace/datos/ecommerce/ecommerce.parquet/")\
               .format("parquet").load(['./datos/ecommerce/ecommerce.parquet/proc='+proc])

# shape:
print(80*'=','\nfilas --> ', df.count(), 'columnas --> ', len(df.columns), '\n', 80*'=')
df.show(truncate=False)

#df = df.withColumn("proc", substring("ctId",1,6))
#df.write.mode("append").partitionBy("proc").parquet("./datos/fakeContracts01.parquet")

filas -->  20692840 columnas -->  10 
+-----------------------+----------------+----------+-------------------+-----------------------+-------+-----+---------+------------------------------------+----------+
|event_time             |event_type      |product_id|category_id        |category_code          |brand  |price|user_id  |user_session                        |proc      |
+-----------------------+----------------+----------+-------------------+-----------------------+-------+-----+---------+------------------------------------+----------+
|2019-11-22 00:00:02 UTC|cart            |5857266   |1487580005134238553|null                   |runail |2.27 |561121542|376106e6-33ae-49a4-8648-e0933ba41c61|2019-11-22|
|2019-11-22 00:00:03 UTC|view            |5877790   |1487580009286598681|null                   |null   |0.43 |574675993|44652247-75d9-47f6-a9b1-b690a799f66d|2019-11-22|
|2019-11-22 00:00:07 UTC|cart            |5857276   |1487580005134238553|null                   |runail |2.22 |5

In [3]:
df.printSchema()

root
 |-- event_time: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)
 |-- proc: date (nullable = true)



### Damos un vistazo rápido al contenido del archivo

In [4]:
df.describe().show()

+-------+--------------------+----------+------------------+--------------------+-------------------+--------+-----------------+-------------------+--------------------+
|summary|          event_time|event_type|        product_id|         category_id|      category_code|   brand|            price|            user_id|        user_session|
+-------+--------------------+----------+------------------+--------------------+-------------------+--------+-----------------+-------------------+--------------------+
|  count|            20692840|  20692840|          20692840|            20692840|             353594|11935723|         20692840|           20692840|            20688242|
|   mean|                null|      null| 5484296.739534883|1.554230182681876...|               null|    null|8.534735480476865|5.215526635599699E8|                null|
| stddev|                null|      null|1305715.7311086198|1.691037849175161...|               null|    null|19.38142460687468|8.744312150200894E7|  

In [5]:
%%time
for c in df.columns:df.groupBy(c).count().show()

+--------------------+-----+
|          event_time|count|
+--------------------+-----+
|2019-11-22 00:05:...|    1|
|2019-11-22 00:08:...|    2|
|2019-11-22 00:10:...|    1|
|2019-11-22 00:12:...|    1|
|2019-11-22 00:37:...|    1|
|2019-11-22 01:04:...|    2|
|2019-11-22 01:23:...|    1|
|2019-11-22 01:23:...|    1|
|2019-11-22 01:24:...|    2|
|2019-11-22 02:17:...|    1|
|2019-11-22 02:30:...|    2|
|2019-11-22 02:31:...|    2|
|2019-11-22 02:36:...|    1|
|2019-11-22 02:42:...|    1|
|2019-11-22 02:52:...|    1|
|2019-11-22 02:54:...|    1|
|2019-11-22 02:55:...|    1|
|2019-11-22 02:55:...|    1|
|2019-11-22 03:03:...|    1|
|2019-11-22 03:11:...|    1|
+--------------------+-----+
only showing top 20 rows

+----------------+-------+
|      event_type|  count|
+----------------+-------+
|        purchase|1287007|
|            view|9657821|
|            cart|5768333|
|remove_from_cart|3979679|
+----------------+-------+



+----------+-----+
|product_id|count|
+----------+-----+
|   5859412| 3365|
|   5870198|   65|
|   5652853|  448|
|   5898955|  144|
|   5809871|11584|
|   5670001| 2969|
|   5744098| 1233|
|   5670719|  549|
|   5762876|  105|
|   5742957| 6968|
|   5826669|  465|
|   5876926| 2891|
|   5657304|  743|
|   5866963| 1244|
|   5847579|  485|
|   5814540|  333|
|   5809880| 2374|
|   5902148| 2465|
|   5767927| 2398|
|   5723483|  408|
+----------+-----+
only showing top 20 rows



+-------------------+------+
|        category_id| count|
+-------------------+------+
|1487580008145748965|277540|
|1487580011677352062| 38218|
|2145935122136826354|  2194|
|1998040852064109417|  9221|
|1487580007004898224|  1128|
|2193074740686488401| 10004|
|1487580005293622112|  3559|
|1924049287554007223|   821|
|1487580007575323592| 64188|
|2055161088059638328| 32175|
|2196456817758831535|  4131|
|1487580009362096156| 37652|
|2130081478220972046|  4101|
|1542195323827388674| 48558|
|1783999068867920626| 36352|
|1487580005411062629|108440|
|1487580005595612013|789465|
|1487580013900333275|  6938|
|1487580012482658454|    74|
|1487580008070251489|  5384|
+-------------------+------+
only showing top 20 rows



+--------------------+--------+
|       category_code|   count|
+--------------------+--------+
|furniture.living_...|   30560|
| stationery.cartrige|   59728|
|                null|20339246|
|furniture.bathroo...|   23931|
|     accessories.bag|   24253|
|appliances.person...|    5384|
|appliances.enviro...|     708|
|        sport.diving|       4|
|appliances.enviro...|  149168|
|furniture.living_...|     343|
|accessories.cosme...|    3644|
|       apparel.glove|   52575|
|appliances.person...|    3296|
+--------------------+--------+



+------------+------+
|       brand| count|
+------------+------+
|     beautix| 99456|
|     farmona|  7903|
|  dr.gloderm|  2162|
|   profhenna|  7193|
|    bioderma|   567|
|     philips|   178|
|       riche|  3260|
|        nova|    14|
|        oniq|116305|
|    lebelage|  4338|
|     vilenta|  4956|
|       fancy|  1531|
|      jaguar|  9135|
|    siberina| 12936|
|      tertio|  7807|
|   koreatida|   752|
|         jas| 27025|
|rocknailstar|   141|
|   depilflax| 28474|
|protokeratin|  3443|
+------------+------+
only showing top 20 rows



+------+------+
| price| count|
+------+------+
|  2.86|211583|
|  13.4|    26|
| 12.32|   713|
| 10.65|   131|
| 19.98|   516|
|  9.13|  3111|
|  8.51|  1664|
| 45.43|   105|
| 17.56|  1163|
| 41.89|    60|
| 17.52|    24|
| 17.95|    48|
|  14.9|    29|
|  26.7|   312|
| 43.03|   116|
|  15.4| 10618|
| 83.17|    54|
|  5.86|  7249|
|150.79|  3788|
|109.19|     4|
+------+------+
only showing top 20 rows



+---------+-----+
|  user_id|count|
+---------+-----+
|462343203|   21|
|601744209|    1|
|601754612|    1|
|600902838|    2|
|598306470|  141|
|518313134|   75|
|601764740|    1|
|601766788|    1|
|477202275|   35|
|495134205|  826|
|479913443|  370|
|588164824|  268|
|601711237|    7|
|601780212|    1|
|601782928|    5|
|573815583|    1|
|561656117|  117|
|592746957|    7|
|595624197|    2|
|596236891|   39|
+---------+-----+
only showing top 20 rows



+--------------------+-----+
|        user_session|count|
+--------------------+-----+
|d44ab980-5a1c-4f0...|    1|
|1565ef5e-c4e5-43a...|    2|
|387a08b9-0a8d-4c3...|    1|
|12a41d5f-c61b-4e5...|    1|
|717d173e-a98c-488...|   78|
|a809ce54-6f79-450...|    1|
|c404afb3-fc22-4d2...|    1|
|684c9489-19ed-412...|    1|
|7b200413-f13e-43f...|    6|
|97ee0d24-72a5-4c4...|    1|
|41525fc3-d50b-414...|   16|
|7b19c3b0-6cde-41e...|    1|
|a80dd2dd-1f56-411...|    1|
|1e84dae5-5af9-4d9...|    1|
|a18301d7-2122-495...|    2|
|7536d637-fc7c-4f8...|    2|
|a481a6eb-60d9-4cc...|    1|
|8c632e1e-447a-472...|    1|
|ffa04015-897b-4b3...|    2|
|4a72db04-a67b-423...|    1|
+--------------------+-----+
only showing top 20 rows

+----------+------+
|      proc| count|
+----------+------+
|2020-02-04|159671|
|2020-02-12|163754|
|2020-02-25|157952|
|2020-01-13|160363|
|2020-01-28|184379|
|2020-02-10|158514|
|2020-02-05|160977|
|2020-01-15|157848|
|2020-02-17|154544|
|2020-02-02|156196|
|2019-11-07|190194

### Más pruebas

In [6]:
df.sort(F.col('price').desc()).show()

+--------------------+----------------+----------+-------------------+-------------+------+------+---------+--------------------+----------+
|          event_time|      event_type|product_id|        category_id|category_code| brand| price|  user_id|        user_session|      proc|
+--------------------+----------------+----------+-------------------+-------------+------+------+---------+--------------------+----------+
|2020-02-16 20:49:...|            view|   5906221|1487580006300255120|         null|strong|327.78|606488825|07fbd8c7-fc97-466...|2020-02-16|
|2020-02-08 18:17:...|            view|   5906221|1487580006300255120|         null|strong|327.78|606865876|d52d3c1b-8248-4f6...|2020-02-08|
|2020-02-16 20:50:...|            view|   5906221|1487580006300255120|         null|strong|327.78|606488825|07fbd8c7-fc97-466...|2020-02-16|
|2020-02-16 16:29:...|            view|   5906221|1487580006300255120|         null|strong|327.78|615886831|8a96b370-b5f8-4f6...|2020-02-16|
|2020-02-08 1

In [7]:
%%time
spark.sparkContext.stop()


CPU times: user 2.06 ms, sys: 135 µs, total: 2.2 ms
Wall time: 582 ms
